In [2]:
import os
import boto3
from botocore.client import Config

# --- IMPORTANT: Set your DigitalOcean credentials and Space information ---
os.environ['SPACES_ACCESS_KEY'] = 'DO801NGY2P92FAUZUQJH'
os.environ['SPACES_SECRET_KEY'] = 'H0uQz+d/Vjbek70S9YcT84iq9sMEnGaHPEI5H3ydv58'

# The endpoint URL for your Space (e.g., nyc3.digitaloceanspaces.com)
# You can find this in your Space's settings.
SPACES_ENDPOINT_URL = 'https://assip2025storage.nyc3.digitaloceanspaces.com' # Change to your region
SPACES_BUCKET_NAME = 'assip2025storage' # Change to your Space's name

In [3]:
import transformers
from pathlib import Path
import shutil

In [ ]:
'''


# --- Configuration (from Step 2) ---
# Ensure your credentials are set as environment variables for security
SPACES_ACCESS_KEY = os.getenv('SPACES_ACCESS_KEY')
SPACES_SECRET_KEY = os.getenv('SPACES_SECRET_KEY')

# --- Function to Upload a Directory ---
def upload_directory_to_space(local_directory, bucket_name, spaces_endpoint_url, access_key, secret_key):
    """
    Uploads the contents of a directory to a DigitalOcean Space.

    :param local_directory: Path to the local directory.
    :param bucket_name: Name of the DigitalOcean Space (bucket).
    """
    print(f"Connecting to DigitalOcean Space: {bucket_name}")
    session = boto3.session.Session()
    client = session.client('s3',
                            region_name=spaces_endpoint_url.split('.')[1], # e.g., 'ams3'
                            endpoint_url=spaces_endpoint_url,
                            aws_access_key_id=access_key,
                            aws_secret_access_key=secret_key)
    
    print(f"Starting upload from '{local_directory}'...")
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            local_path = os.path.join(root, filename)
            # Create a relative path for the object in the Space
            relative_path = os.path.relpath(local_path, local_directory)
            # On Windows, path separators are '\', S3 uses '/'.
            s3_path = relative_path.replace("\\", "/")

            print(f"Uploading {local_path} to s3://{bucket_name}/{s3_path}")
            try:
                client.upload_file(local_path, bucket_name, s3_path)
            except Exception as e:
                print(f"Error uploading {local_path}: {e}")
                return False
    print("Upload complete.")
    return True

# --- Main Script ---

# 1. Define the model and a temporary local cache directory
model_name = "EleutherAI/pythia-1.4b-deduped" # Example model
local_temp_cache = Path("./temp_model_cache")
local_temp_cache.mkdir(exist_ok=True) # Create the directory if it doesn't exist

print(f"Downloading model '{model_name}' to temporary local cache: {local_temp_cache}")

# 2. Download the model using the transformers library
# The `cache_dir` argument tells Hugging Face where to save the files.
try:
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_name,
        cache_dir=str(local_temp_cache)
    )
    print("Model download complete.")

    # 3. Upload the entire cache directory to DigitalOcean Spaces
    if upload_directory_to_space(str(local_temp_cache), SPACES_BUCKET_NAME, SPACES_ENDPOINT_URL, SPACES_ACCESS_KEY, SPACES_SECRET_KEY):
        # 4. Clean up the local directory to free space
        print(f"Successfully uploaded to Spaces. Deleting local directory: {local_temp_cache}")
        shutil.rmtree(local_temp_cache)
        print("Local cache deleted.")
    else:
        print("Upload failed. Local files have not been deleted.")

except Exception as e:
    print(f"An error occurred during download: {e}")

'''

In [4]:
from huggingface_hub import login
login()

In [5]:
from datasets import load_dataset
import json
from transformers import TextStreamer



In [6]:
ds = load_dataset("cat-searcher/minif2f-lean4", split="test")

README.md:   0%|          | 0.00/212 [00:00<?, ?B/s]

c:\Users\mlevi\miniconda3\envs\assip2025\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mlevi\.cache\huggingface\hub\datasets--cat-searcher--minif2f-lean4. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


valid.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating validation split:   0%|          | 0/244 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/244 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "RickyDeSkywalker/TheoremLlama"
tok  = AutoTokenizer.from_pretrained(model_id, cache_dir=CACHE_DIR)
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            quantization_config=bnb_config,
            device_map="auto",            # Colab T4 / A10 will work
            cache_dir=CACHE_DIR
        )
streamer = TextStreamer(tok)             # prints streaming tokens for visibility
